In [4]:
import os
import cv2
import shutil
import numpy as np

from skimage import io
from pathlib import Path

In [5]:
# Number of extra pairs for each image
nb_extra_pairs = 5

# Paths of input folders and output folders (from current working directory)
# Input folder should consist of an even number of images, where always 2 pictures create a pair
# 1st picture will be moved, 2nd will remain static
path_imgs_in = "Images"
path_imgs_out = "Images_rotated"
# New ground truths will be created (by rotating first point like picture) so that evaluation is possible
path_gts_in = "Ground Truth"
path_gts_out = "ground_truths_rotated"

# Creates output directories if they don't yet exist
Path(path_imgs_out).mkdir(parents=True, exist_ok=True)
Path(path_gts_out).mkdir(parents=True, exist_ok=True)

# Maximum allowed rotation for the first picture, value for both directions [-max_rot, max_rot)
max_rot = 30

In [6]:
# Get input folder of image pairs
image_names = os.listdir(os.path.join(os.getcwd(), path_imgs_in))

# Use every second index to get pairs
for j in range(0, len(image_names), 2):
    # VARIABLES
    
    # Get Names of Images
    image_name1 = image_names[j]
    image_name2 = image_names[j+1]
    
    # Get Groundtruth name
    pair_name = image_name1[:3] 
    gt_path = os.path.join(os.getcwd(), path_gts_in) 
    gt_name = "control_points_" + pair_name + "_1_2.txt"
    
    # Do it nb_extra_pairs times for each image pair in the in-folder
    for i in range(nb_extra_pairs):
        # Create random rotation
        rotation = round(np.random.rand()*2*max_rot - max_rot)

        # Change output names of files to have rotation degree in it for distinction
        # Naming convention: Pairname _ rotDEGREE _ index-of-img-in-pair . fileending
        # rsplit here splits at the last ".", [:-2] is to cut off the index-of-img-in-pair
        im1_name_out = image_name1.rsplit(".", 1)[0][:-2] +"_rot"+ str(rotation) + "_1.jpg"
        im2_name_out = image_name2.rsplit(".", 1)[0][:-2] +"_rot"+ str(rotation) + "_2.jpg"
        gt_name_out = gt_name.rsplit(".", 1)[0] +"_rot"+ str(rotation) +".txt"     


        # IMAGE

        # Load image and rotate
        im1 = io.imread(os.path.join(path_imgs_in, image_name1))
        # normalize if nessecary
        if np.max(im1) > 1:
            im1 = im1 / 255  
        # get rotation matrix with center in the middle of the first image
        rotate_matrix = cv2.getRotationMatrix2D(center=(im1.shape[0]//2, im1.shape[1]//2), angle=rotation, scale=1)
        im1_rot = cv2.warpAffine(src=im1, M=rotate_matrix, dsize=(im1.shape[0], im1.shape[1]))

        # Converse image to uint8 for saving 
        if np.max(im1_rot) <= 1:
            im1_rot *= 256
        im1_rot = im1_rot.astype("uint8")
        io.imsave(os.path.join(os.getcwd(), path_imgs_out, im1_name_out), im1_rot)

        # Load the fixed image and save it in new folder as well, so that pairs are together
        im2 = io.imread(os.path.join(path_imgs_in, image_name2))
        io.imsave(os.path.join(os.getcwd(), path_imgs_out, im2_name_out), im2)


        # GROUND TRUTHS

        # Read the ground truths and rotate first point accordingly just like the image
        # Write new ground truths into the gt_out folder
        with open(os.path.join(path_gts_in, gt_name), "r") as f_in:  
            with open(os.path.join(os.getcwd(), path_gts_out, gt_name_out), "w") as f_out:                     
                lines = f_in.readlines()  
                for line in lines:
                    # Read the values in the file
                    values = [float(x) for x in line[:-1].split(" ")]
                    # first 2 values are point on the rotated image
                    point1 = np.asarray([[values[0], values[1]]])
                    # rotate these points as well
                    point1_rot = np.matmul(point1, rotate_matrix)[0, :2]

                    # 1st point rotated, 2nd point the same
                    f_out.write(f"{round(point1_rot[0])} {round(point1_rot[1])} {values[2]} {values[3]}\n")  
        
        print(f"Finished {i+1}/{nb_extra_pairs} for pair {pair_name}")
    print(f"Pair {pair_name} done. {j//2 +1}/{len(image_names)//2}\n")

Finished 1/5 for pair A01
Finished 2/5 for pair A01
Finished 3/5 for pair A01
Finished 4/5 for pair A01
Finished 5/5 for pair A01
Pair A01 done. 1/134

Finished 1/5 for pair A02
Finished 2/5 for pair A02
Finished 3/5 for pair A02
Finished 4/5 for pair A02
Finished 5/5 for pair A02
Pair A02 done. 2/134

Finished 1/5 for pair A03
Finished 2/5 for pair A03
Finished 3/5 for pair A03
Finished 4/5 for pair A03
Finished 5/5 for pair A03
Pair A03 done. 3/134

Finished 1/5 for pair A04
Finished 2/5 for pair A04
Finished 3/5 for pair A04
Finished 4/5 for pair A04
Finished 5/5 for pair A04
Pair A04 done. 4/134

Finished 1/5 for pair A05
Finished 2/5 for pair A05
Finished 3/5 for pair A05
Finished 4/5 for pair A05
Finished 5/5 for pair A05
Pair A05 done. 5/134

Finished 1/5 for pair A06
Finished 2/5 for pair A06
Finished 3/5 for pair A06
Finished 4/5 for pair A06
Finished 5/5 for pair A06
Pair A06 done. 6/134

Finished 1/5 for pair A07
Finished 2/5 for pair A07
Finished 3/5 for pair A07
Finished 4

Finished 5/5 for pair P40
Pair P40 done. 54/134

Finished 1/5 for pair P41
Finished 2/5 for pair P41
Finished 3/5 for pair P41
Finished 4/5 for pair P41
Finished 5/5 for pair P41
Pair P41 done. 55/134

Finished 1/5 for pair P42
Finished 2/5 for pair P42
Finished 3/5 for pair P42
Finished 4/5 for pair P42
Finished 5/5 for pair P42
Pair P42 done. 56/134

Finished 1/5 for pair P43
Finished 2/5 for pair P43
Finished 3/5 for pair P43
Finished 4/5 for pair P43
Finished 5/5 for pair P43
Pair P43 done. 57/134

Finished 1/5 for pair P44
Finished 2/5 for pair P44
Finished 3/5 for pair P44
Finished 4/5 for pair P44
Finished 5/5 for pair P44
Pair P44 done. 58/134

Finished 1/5 for pair P45
Finished 2/5 for pair P45
Finished 3/5 for pair P45
Finished 4/5 for pair P45
Finished 5/5 for pair P45
Pair P45 done. 59/134

Finished 1/5 for pair P46
Finished 2/5 for pair P46
Finished 3/5 for pair P46
Finished 4/5 for pair P46
Finished 5/5 for pair P46
Pair P46 done. 60/134

Finished 1/5 for pair P47
Finishe

Finished 3/5 for pair S45
Finished 4/5 for pair S45
Finished 5/5 for pair S45
Pair S45 done. 108/134

Finished 1/5 for pair S46
Finished 2/5 for pair S46
Finished 3/5 for pair S46
Finished 4/5 for pair S46
Finished 5/5 for pair S46
Pair S46 done. 109/134

Finished 1/5 for pair S47
Finished 2/5 for pair S47
Finished 3/5 for pair S47
Finished 4/5 for pair S47
Finished 5/5 for pair S47
Pair S47 done. 110/134

Finished 1/5 for pair S48
Finished 2/5 for pair S48
Finished 3/5 for pair S48
Finished 4/5 for pair S48
Finished 5/5 for pair S48
Pair S48 done. 111/134

Finished 1/5 for pair S49
Finished 2/5 for pair S49
Finished 3/5 for pair S49
Finished 4/5 for pair S49
Finished 5/5 for pair S49
Pair S49 done. 112/134

Finished 1/5 for pair S50
Finished 2/5 for pair S50
Finished 3/5 for pair S50
Finished 4/5 for pair S50
Finished 5/5 for pair S50
Pair S50 done. 113/134

Finished 1/5 for pair S51
Finished 2/5 for pair S51
Finished 3/5 for pair S51
Finished 4/5 for pair S51
Finished 5/5 for pair S5